In [1]:
import pandas as pd
import numpy as np
import torch, math, copy, tqdm
from torch import nn, optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import CyclicLR, CosineAnnealingLR
from torch.utils.data import DataLoader, TensorDataset
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import wandb
# from torch inport cosine

In [9]:
wandb.init(
    project='BDISP'
)

In [203]:
dataset_dir = './dataset/'
topics = pd.read_csv(dataset_dir + 'topics.csv')
users = pd.read_csv(dataset_dir + 'users.csv')

In [204]:

users['time'] = pd.to_datetime(users['birthDate'], unit='ms')

users['year'] = pd.DatetimeIndex(users['time']).year
users['month'] = pd.DatetimeIndex(users['time']).month
users['day'] = pd.DatetimeIndex(users['time']).day

users.day.fillna(users.day.median(), inplace=True)
users.month.fillna(users.month.median(), inplace=True)
users.year.fillna(users.year.median(), inplace=True)

cleared_users = users.drop(["time", "birthDate"], axis=1)
cleared_users["year"] = cleared_users.year.astype("int")
cleared_users["month"] = cleared_users.month.astype("int")
cleared_users["day"] = cleared_users.day.astype("int")
cleared_users["age"] = 2023-cleared_users.year
users["age"] = 2023-cleared_users.year
users = users.drop(columns=["time", "birthDate", "year", "month", "day"])

In [205]:
# gender_onehot = pd.get_dummies(users['gender'], prefix='g').drop(columns='g_49')
# users = users.join(gender_onehot).drop(columns=['gender'])

In [206]:
topics = topics.set_index('topicId')

In [207]:
topics['textEmbedding'] = topics[topics.columns[:100]].apply(
    lambda x: np.array(x),
    axis=1
)
topics['imageEmbedding'] = topics[topics.columns[100:-1]].apply(
    lambda x: np.array(x),
    axis=1
)

textembcol = [f'textEmbedding{i}' for i in range(1, 101)]
imgembcol = [f'imageEmbedding{i}' for i in range(1, 513)]
topics = topics.drop(columns=textembcol + imgembcol)

In [208]:
train = pd.read_csv(dataset_dir + 'train.csv')

In [212]:
topic_feedback = train.groupby(['topicId', 'feedback'], as_index=False).count()
topic_feedback

,topicId,feedback,userId
0,0,D,59431
1,0,L,3564
2,1,D,46184
3,1,L,4855
4,2,D,38863
...,...,...,...
334028,206327,L,1
334029,206328,L,1
334030,206329,D,1
334031,206330,D,1


In [213]:
arr = topic_feedback.to_numpy()
d = {}
l = {}
arr
for i, j, k in arr:
    if j == 'D':
        d[i] = k
    elif j == 'L':
        l[i] = k
ar = list(map(lambda x: x[0], arr))
for i in ar:
    if i not in d:
        d[i] = 0
    if i not in l:
        l[i] = 0
        

In [214]:
df_d = pd.DataFrame(pd.Series(d)).rename(columns={0: 'D_topics'})
df_l = pd.DataFrame(pd.Series(l)).rename(columns={0: 'L_topics'})
df_d

,D_topics
0,59431
1,46184
2,38863
3,43114
4,40507
...,...
206324,0
206325,0
206327,0
206328,0


In [215]:
topic_feedback = pd.concat([df_d, df_l], axis=1)
topic_feedback    

,D_topics,L_topics
0,59431,3564
1,46184,4855
2,38863,6579
3,43114,1748
4,40507,3179
...,...,...
206324,0,1
206325,0,1
206327,0,1
206328,0,1


In [216]:
topics = topics.reset_index()
topics = pd.concat([topics.set_index('topicId'), topic_feedback],
          axis=1).reset_index().rename(columns={'index': 'topicId'}).fillna(0)
topics

,topicId,textEmbedding,imageEmbedding,D_topics,L_topics
0,190991,"[0.10322654, 0.11529317, 0.05250684, 0.0736957...","[0.3938745, 1.2072593, 1.3675294, 0.49479333, ...",0,1
1,2718,"[-0.03498603, 0.07697702, 0.01884454, 0.095175...","[0.5260104, 1.9416583, 1.768048, 0.66228265, 0...",483,333
2,49923,"[0.08249106, 0.05636501, 0.03345522, -0.047510...","[1.6287256, 0.6095899, 1.2651148, 1.6559641, 2...",14,15
3,124548,"[0.00930888, 0.05934659, -0.03245487, 0.118811...","[1.3582602, 0.37345153, 1.2514493, 0.79741836,...",0,4
4,45289,"[0.02304623, 0.02378386, -0.03602674, 0.063102...","[0.34933716, 1.6957184, 0.25375903, 2.2800794,...",25,9
...,...,...,...,...,...
206327,30555,"[-0.000419500109, 0.0624226183, 0.0233562589, ...","[0.44671872, 0.8201386, 0.24040394, 0.6394815,...",3,56
206328,111715,"[-0.0543304831, -0.0510341637, -0.0315544829, ...","[0.6109063, 2.1711824, 0.81499237, 1.6031251, ...",3,3
206329,17614,"[0.01303456, 0.01392905, 0.01889999, 0.1482853...","[1.4768978, 3.254887, 2.374536, 1.9267961, 2.1...",47,73
206330,195949,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0567273, 1.4017448, 0.1270614, 0.80723584, ...",0,1


In [217]:
topics['feedback_topics'] = topics[topics.columns[3:5]].apply(
    lambda x: 0.5 if x[0] + x[1] == 0 else (x[1] / (x[0] + x[1])),
    axis=1
)
topics

,topicId,textEmbedding,imageEmbedding,D_topics,L_topics,feedback_topics
0,190991,"[0.10322654, 0.11529317, 0.05250684, 0.0736957...","[0.3938745, 1.2072593, 1.3675294, 0.49479333, ...",0,1,1.000000
1,2718,"[-0.03498603, 0.07697702, 0.01884454, 0.095175...","[0.5260104, 1.9416583, 1.768048, 0.66228265, 0...",483,333,0.408088
2,49923,"[0.08249106, 0.05636501, 0.03345522, -0.047510...","[1.6287256, 0.6095899, 1.2651148, 1.6559641, 2...",14,15,0.517241
3,124548,"[0.00930888, 0.05934659, -0.03245487, 0.118811...","[1.3582602, 0.37345153, 1.2514493, 0.79741836,...",0,4,1.000000
4,45289,"[0.02304623, 0.02378386, -0.03602674, 0.063102...","[0.34933716, 1.6957184, 0.25375903, 2.2800794,...",25,9,0.264706
...,...,...,...,...,...,...
206327,30555,"[-0.000419500109, 0.0624226183, 0.0233562589, ...","[0.44671872, 0.8201386, 0.24040394, 0.6394815,...",3,56,0.949153
206328,111715,"[-0.0543304831, -0.0510341637, -0.0315544829, ...","[0.6109063, 2.1711824, 0.81499237, 1.6031251, ...",3,3,0.500000
206329,17614,"[0.01303456, 0.01392905, 0.01889999, 0.1482853...","[1.4768978, 3.254887, 2.374536, 1.9267961, 2.1...",47,73,0.608333
206330,195949,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0567273, 1.4017448, 0.1270614, 0.80723584, ...",0,1,1.000000


In [218]:
user_feedback = train.groupby(['userId', 'feedback'], as_index=False).count()
user_feedback

,userId,feedback,topicId
0,0,D,276
1,0,L,661
2,1,D,265
3,1,L,609
4,2,D,367
...,...,...,...
5173518,2682252,L,1
5173519,2682253,L,1
5173520,2682254,L,1
5173521,2682255,L,1


In [219]:
arr = user_feedback.to_numpy()
d = {}
l = {}
arr
for i, j, k in arr:
    if j == 'D':
        d[i] = k
    elif j == 'L':
        l[i] = k
ar = list(map(lambda x: x[0], arr))
for i in ar:
    if i not in d:
        d[i] = 0
    if i not in l:
        l[i] = 0
        

In [220]:
df_d = pd.DataFrame(pd.Series(d)).rename(columns={0: 'D_users'})
df_l = pd.DataFrame(pd.Series(l)).rename(columns={0: 'L_users'})
df_d


,D_users
0,276
1,265
2,367
3,342
4,619
...,...
2682252,0
2682253,0
2682254,0
2682255,0


In [221]:
user_feedback = pd.concat([df_d, df_l], axis=1)
user_feedback    

,D_users,L_users
0,276,661
1,265,609
2,367,449
3,342,430
4,619,89
...,...,...
2682252,0,1
2682253,0,1
2682254,0,1
2682255,0,1


In [222]:
users = pd.concat([users.set_index('userId'), user_feedback],
          axis=1).reset_index().rename(columns={'index': 'userId'}).fillna(0)
users

,userId,gender,locationId,age,D_users,L_users
0,2019458,2,5697551,43,1,1
1,96675,2,12141127,71,12,12
2,1378979,1,12141421,39,1,1
3,540187,1,4126402,41,3,5
4,1182831,2,2955426,53,2,2
...,...,...,...,...,...,...
2682252,1362388,1,4155306,73,1,1
2682253,470190,2,2117251,47,4,4
2682254,608546,2,2784685,53,2,4
2682255,1559056,2,2267414,58,1,1


In [223]:
users['feedback_users'] = users[users.columns[4:6]].apply(
    lambda x: 0.5 if x[0] + x[1] == 0 else (x[1] / (x[0] + x[1])),
    axis=1
)
users

,userId,gender,locationId,age,D_users,L_users,feedback_users
0,2019458,2,5697551,43,1,1,0.500000
1,96675,2,12141127,71,12,12,0.500000
2,1378979,1,12141421,39,1,1,0.500000
3,540187,1,4126402,41,3,5,0.625000
4,1182831,2,2955426,53,2,2,0.500000
...,...,...,...,...,...,...,...
2682252,1362388,1,4155306,73,1,1,0.500000
2682253,470190,2,2117251,47,4,4,0.500000
2682254,608546,2,2784685,53,2,4,0.666667
2682255,1559056,2,2267414,58,1,1,0.500000


In [224]:
active_id_count = train.userId.value_counts()
users['active'] = users.userId.apply(lambda x: active_id_count.get(x))
users['active'] = users['active'].fillna(0)
users

,userId,gender,locationId,age,D_users,L_users,feedback_users,active
0,2019458,2,5697551,43,1,1,0.500000,2
1,96675,2,12141127,71,12,12,0.500000,24
2,1378979,1,12141421,39,1,1,0.500000,2
3,540187,1,4126402,41,3,5,0.625000,8
4,1182831,2,2955426,53,2,2,0.500000,4
...,...,...,...,...,...,...,...,...
2682252,1362388,1,4155306,73,1,1,0.500000,2
2682253,470190,2,2117251,47,4,4,0.500000,8
2682254,608546,2,2784685,53,2,4,0.666667,6
2682255,1559056,2,2267414,58,1,1,0.500000,2


In [225]:
from sklearn import preprocessing

In [226]:
users['active'] = preprocessing.normalize([users['active'].to_numpy()])[0]

In [227]:
users['age'] = preprocessing.normalize([users['age'].to_numpy()])[0]

In [228]:
train['feedback'] = train.feedback.apply(lambda x: int(x == 'L'))
# train = prepare(xtrain)

In [229]:
def parse_embeddings(frame: pd.DataFrame):
    arrays = []
    for arr in frame.to_numpy():
        arrays.append(arr)
    return np.array(arrays)

In [230]:
vals = users.locationId.values
vals = list(set(vals))
vals.sort()
def lower_bound(x):
    l, r = 0, len(vals)
    while l < r - 1:
        m = (r + l) // 2
        if vals[m] > x:
            r = m
        else:
            l = m
    return l
users.locationId = users.locationId.apply(lower_bound)
# vals
users


,userId,gender,locationId,age,D_users,L_users,feedback_users,active
0,2019458,2,82097,0.000473,1,1,0.500000,0.000106
1,96675,2,101439,0.000782,12,12,0.500000,0.001268
2,1378979,1,101524,0.000429,1,1,0.500000,0.000106
3,540187,1,68968,0.000451,3,5,0.625000,0.000423
4,1182831,2,49278,0.000583,2,2,0.500000,0.000211
...,...,...,...,...,...,...,...,...
2682252,1362388,1,69456,0.000804,1,1,0.500000,0.000106
2682253,470190,2,35212,0.000517,4,4,0.500000,0.000423
2682254,608546,2,46428,0.000583,2,4,0.666667,0.000317
2682255,1559056,2,37700,0.000638,1,1,0.500000,0.000106


In [231]:
def prepare(df: pd.DataFrame) -> pd.DataFrame:
    _ = df.set_index('userId').join(users.set_index('userId')).reset_index()
    _ = _.set_index('topicId').join(topics.set_index('topicId')).reset_index()
    _['gender'] = _['gender'].apply(lambda x: 0 if x == 49 else x)
    return _

In [232]:
from itertools import zip_longest

class Model(nn.Module):
    def __init__(
        self,
        topics: pd.DataFrame,
        users: pd.DataFrame,
        hidden: list,
        dropouts: list,
        embedding_dropout=0.02,
        user_features: int = 5
        ):
        super(Model, self).__init__()
        
        n_last = hidden[-1]
        
        self.emb_img = nn.Embedding(topics.shape[0], 512)
        self.emb_text = nn.Embedding(topics.shape[0], 100)
        
        self.ifc = nn.Linear(512, 128)
        self.tfc = nn.Linear(100, 32)
        
        self.relu = nn.ReLU()
        self.flat = nn.Flatten()
        
        self.emb = nn.Embedding(users.shape[0], user_features)
    
        # self.ufc = nn.Linear(user_features + 4, user_features)
        
        im_embeddings = parse_embeddings(topics.sort_index()['imageEmbedding'])
        tx_embeddings = parse_embeddings(topics.sort_index()['textEmbedding'])
        
        self.emb_img.weight.data.copy_(torch.from_numpy(im_embeddings))
        self.emb_text.weight.data.copy_(torch.from_numpy(tx_embeddings))

        self.drop = nn.Dropout(embedding_dropout)
        
        def gen_layers(n_in):
            """
            A generator that yields a sequence of hidden layers and 
            their activations/dropouts.
            
            Note that the function captures `hidden` and `dropouts` 
            values from the outer scope.
            """
            nonlocal hidden, dropouts
            assert len(dropouts) <= len(hidden)
            
            for n_out, rate in zip_longest(hidden, dropouts):
                yield nn.Linear(n_in, n_out)
                yield nn.ReLU()
                if rate is not None and rate > 0.:
                    yield nn.Dropout(rate)
                n_in = n_out
        
        self.hidden = nn.Sequential(*list(gen_layers(128 + 32 + user_features * 4 + 4)))
        
        self.fc = nn.Linear(n_last, 1)
        self._init()
        
    def forward(self, x, minmax=None):
        users = x[:, 0].type(torch.int32)
        topics = x[:, 1].type(torch.int32)
        user_features = x[:, 2:4].type(torch.int32)
        custom = x[:, 4:]
        
        # user_features = x[:, 2:].float()
        user_features = self.flat(self.emb(user_features))
        im = self.emb_img(topics)
        tx = self.emb_text(topics)
        
        im = self.relu(self.ifc(im))
        tx = self.relu(self.tfc(tx))
        custom = custom.type(tx.dtype)
        # us = self.ufc(torch.cat([us, user_features], dim=1))
        features = torch.cat([user_features, im, tx, self.emb(users), self.emb(topics), custom], dim=1)
        x = self.drop(features)
        # x = self.hidden(torch.cat([x, user_features], dim=1))
        x = self.hidden(x)
        out = torch.sigmoid(self.fc(x))
        if minmax is not None:
            min_rating, max_rating = minmax
            out = out*(max_rating - min_rating + 1) + min_rating - 0.5
        return out
    
    def _init(self):
        """
        Setup embeddings and hidden layers with reasonable initial values.
        """
        
        def init(m):
            ...
            # if type(m) == nn.Linear:
            #     torch.nn.init.xavier_uniform_(m.weight)
            #     m.bias.data.fill_(0.01)
                
        # self.im.weight.data.uniform_(-0.05, 0.05)
        # self.tx.weight.data.uniform_(-0.05, 0.05)
        # self.us.weight.data.uniform_(-0.05, 0.05)
        self.hidden.apply(init)
        init(self.fc)

In [233]:
identifier = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(identifier)
device

device(type='cuda', index=0)

In [258]:
net = Model(
    topics=topics,
    users=users,
    hidden=[256, 256, 128, 32],
    dropouts=[0.05, 0.1]
)

In [324]:
lr = 1e-5
wd = 1e-5
bs = 4096 * 2
n_epochs = 10
patience = 10
no_improvements = 0
best_loss = np.inf
best_weights = None
history = []
lr_history = []




In [236]:
prepare(train).head()

,topicId,userId,feedback,gender,locationId,age,D_users,L_users,feedback_users,active,textEmbedding,imageEmbedding,D_topics,L_topics,feedback_topics
0,0,2,0,2,37598,0.000319,367,449,0.550245,0.043118,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
1,0,3,1,2,8859,0.000749,342,430,0.556995,0.040793,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
2,0,6,0,2,79588,0.000528,347,315,0.475831,0.034981,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
3,0,7,0,2,13891,0.000506,207,449,0.684451,0.034664,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
4,0,10,0,1,143121,0.000440,227,360,0.613288,0.031018,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576


In [254]:
data1 = torch.from_numpy(t[['userId', 'topicId']].to_numpy())
data2 = torch.from_numpy(prepare(t)[['gender', 'locationId', 'age', 'active', 'feedback_users', 'feedback_topics']].to_numpy())
x_train, x_test, y_train, y_test = \
    train_test_split(
        # data1,
        torch.cat([data1, data2], dim=1),
        torch.from_numpy(t[['feedback']].to_numpy(dtype='float32')),
        test_size=0.05,
        shuffle=1
        )

In [255]:
train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=bs, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=bs, shuffle=False)

In [256]:
def macro_double_soft_f1(y, y_hat, reduction='mean'): # Written in PyTorch
    tp = (y_hat * y).sum(dim=0) # soft
    fp = (y_hat * (1 - y)).sum(dim=0) # soft
    fn = ((1 - y_hat) * y).sum(dim=0) # soft
    tn = ((1 - y_hat) * (1 - y)).sum(dim=0) # soft

    soft_f1_class1 = 2 * tp / (2 * tp + fn + fp + 1e-16)
    soft_f1_class0 = 2 * tn / (2 * tn + fn + fp + 1e-16)
    cost_class1 = 1 - soft_f1_class1 # reduce 1 - soft-f1_class1 in order to increase soft-f1 on class 1
    cost_class0 = 1 - soft_f1_class0 # reduce 1 - soft-f1_class0 in order to increase soft-f1 on class 0
    cost = 0.5 * (cost_class1 + cost_class0) # take into account both class 1 and class 0

    if reduction == 'none':
        return cost

    if reduction == 'mean':
        macro_cost = cost.mean()
        return macro_cost

In [261]:
net.to(device)
# criterion = nn.BCEWithLogitsLoss()
criterion = macro_double_soft_f1
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
iterations_per_epoch = int(math.ceil(x_train.shape[0] // bs))
scheduler = CosineAnnealingLR(optimizer, T_max=iterations_per_epoch * 2, eta_min=lr/10)

In [262]:
fmt = '[{epoch:03d}/{total:03d}] train: {train:.4f} - val: {val:.4f}'

# start training
for epoch in range(n_epochs):
    stats = {'epoch': epoch + 1, 'total': n_epochs}
    
    for phase in ('train', 'val'):
        training = phase == 'train'
        length = len(train_loader) if training else len(test_loader)
        running_loss = 0.0
        n_batches = 0
        
        with tqdm.tqdm((test_loader, train_loader)[training], unit="batch", mininterval=0) as bar:
            bar.set_description(f"Epoch {epoch}")
            
            for batch in bar:
                x_batch, y_batch = [b.to(device) for b in batch]
                # x_batch, y_batch = batch
                optimizer.zero_grad()
            
                with torch.set_grad_enabled(training):
                    outputs = net(x_batch)
                    loss = criterion(outputs, y_batch)
                    if training:
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                        lr_history.extend(scheduler.get_last_lr())
                        
                running_loss += loss.item()
                
                acc = f1_score(y_batch.cpu().detach().numpy().reshape(-1), outputs.cpu().round().detach().numpy().reshape(-1), average='binary')
                wandb.log({'acc': acc, 'loss': loss.item(), 'lr': scheduler.get_last_lr()[-1]})
                
                bar.set_postfix(
                    loss=float(loss.item()),
                    acc=float(acc),
                )
                
        epoch_loss = running_loss / length
        stats[phase] = epoch_loss
        if phase == 'val':
            if epoch_loss < best_loss:
                print('loss improvement on epoch: %d' % (epoch + 1))
                best_loss = epoch_loss
                best_weights = copy.deepcopy(net.state_dict())
                no_improvements = 0
            else:
                no_improvements += 1
                
    history.append(stats)
    print(fmt.format(**stats))

    if no_improvements >= patience:
        break
wandb.finish()

Epoch 0: 100%|██████████| 2/2 [00:00<00:00, 32.01batch/s, acc=0.712, loss=0.5]


loss improvement on epoch: 1
[001/010] train: 0.5015 - val: 0.4999


Epoch 1: 100%|██████████| 2/2 [00:00<00:00, 32.01batch/s, acc=0.712, loss=0.5]  


loss improvement on epoch: 2
[002/010] train: 0.4995 - val: 0.4996


Epoch 2: 100%|██████████| 2/2 [00:00<00:00, 32.01batch/s, acc=0.712, loss=0.499]


...

Epoch 8: 100%|██████████| 2/2 [00:00<00:00,  9.14batch/s, acc=0.656, loss=0.383]


loss improvement on epoch: 9
[009/010] train: 0.3006 - val: 0.3831


Epoch 9: 100%|██████████| 2/2 [00:00<00:00, 32.01batch/s, acc=0.634, loss=0.388]

[010/010] train: 0.2866 - val: 0.3888


acc,▇█▁█▇▆██████████▇██▇█▇██▇▅▇▇▇▂▇▇▇▇█▇▇▃▆▅
loss,▂▂█▁▂▅▁▁▁▁▁▁▁▁▂▁▂▁▁▂▁▂▁▁▂█▅▄▄█▂▂▂▂▁▂▄▇▄▇
lr,▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂▂▂███▇▄▄▄▄▃▃▃▇▄▃▄
acc,0.63389
loss,0.38845
lr,0.0001


In [ ]:
loaded_net = torch.load(open('model20.pt', 'rb'))

In [263]:
net.eval()
yt_batch = []
preds = []
with tqdm.tqdm(test_loader, unit="batch", mininterval=0) as bar:
    # bar.set_description(f"Epoch {epoch}")
    
    for batch in bar:
        # x_batch, y_batch = [b.to(device) for b in batch]
        x_batch, y_batch = batch
        optimizer.zero_grad()
    
        with torch.set_grad_enabled(True):
            outputs = net(x_batch.to(device))
            loss = criterion(outputs, y_batch.to(device))
        running_loss += loss.item()
        
        yt_batch.append(y_batch.cpu().detach().numpy().reshape(-1))
        preds.append(outputs.cpu().round().detach().numpy().reshape(-1))
    acc = f1_score(np.concatenate(yt_batch), np.concatenate(preds), average='binary')
print(acc)

100%|██████████| 2/2 [00:00<00:00, 14.24batch/s]

0.6468859085063384


In [264]:
test = pd.read_csv(dataset_dir + 'test.csv')

In [265]:
result = []
data1 = torch.from_numpy(test[['userId', 'topicId']].to_numpy())
data2 = torch.from_numpy(prepare(test)[['gender', 'locationId', 'age', 'active', 'feedback_users', 'feedback_topics']].to_numpy())
data_loader = DataLoader(torch.cat([data1, data2], dim=1), batch_size=2000)

In [270]:
t = prepare(test)

In [271]:
t

,topicId,userId,gender,locationId,age,D_users,L_users,feedback_users,active,textEmbedding,imageEmbedding,D_topics,L_topics,feedback_topics
0,0,70,2,29416,0.000363,196,162,0.452514,0.018917,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
1,0,179,2,76599,0.000528,128,152,0.542857,0.014796,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
2,0,365,2,5456,0.000749,123,103,0.455752,0.011942,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
3,0,673,2,74815,0.000638,100,86,0.462366,0.009828,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
4,0,831,2,72043,0.000506,76,96,0.558140,0.009089,"[0.02579175, 0.07325585, -0.00653977, 0.106602...","[1.4968897, 0.78598845, 0.4453732, 1.3486475, ...",59431,3564,0.056576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578797,206263,2467,2,8859,0.000782,106,10,0.086207,0.006130,"[0.00256486, 0.01714072, -0.01212274, 0.043945...","[1.6021473, 0.7755553, 0.91788507, 0.5592149, ...",0,1,1.000000
1578798,206271,1950,2,95058,0.001266,55,71,0.563492,0.006658,"[-0.0328400023, 0.0409817249, -0.0611910708, 0...","[1.6514388, 1.4302953, 0.07116745, 1.5843134, ...",0,1,1.000000
1578799,206279,115214,2,8859,0.000396,7,14,0.666667,0.001110,"[0.01195597, 0.05768507, -0.00373112, 0.082402...","[0.061008852, 0.7814988, 0.39236963, 2.693202,...",1,0,0.000000
1578800,206289,6788,2,65042,0.000473,33,47,0.587500,0.004227,"[-0.010609, 0.04301209, -0.07906619, 0.0234069...","[1.0437027, 1.5123883, 0.38676655, 1.5017662, ...",1,0,0.000000


In [317]:
data = t['feedback_users'].to_numpy()
data *= t['active'].to_numpy()
data += t['feedback_topics'].to_numpy()
# data += t['gender'].to_numpy() / 2
data += np.random.rand(len(data)) * 0.1

In [318]:
max(data)

1.1583862470916608

In [266]:
for batch in data_loader:
    batch = batch.to(device)
    pred = net(batch)
    result += list(pred.cpu().detach().numpy().round())

In [321]:
result = []
for i in data:
    if i > 0.6:
        result.append('D')
    else:
        result.append('L')

In [267]:
result = list(map(lambda x: 'DL'[int(x.item())], result))

In [322]:
result.count('L')

838975

In [323]:
open('res1.txt', 'w').write('\n'.join(result))

3157603